### Run cell below to play Boxy!
##### Requires PyGame Module

In [4]:
import pygame

pygame.init()

# Standard modules
import numpy as np
#from time import sleep

# Custom 
from Super_Classes import Body
from Status import Status
from Player import Player
from Constants import display_size
from Level import Level
from Make_Sounds import ouch_sound, thud_sound
from Keyboard import Keyboard



gameDisplay = pygame.display.set_mode(display_size)
pygame.display.set_caption('Game!')

sky = (205, 230, 247)
clock = pygame.time.Clock()


# Object to determine what to draw on screen
screen = Body([0,0],display_size,corporeal=True,solid=False,velocity=[0,0])   

# Object holding all level Bodies and scenery
level = Level(num=0)


death_delay = 60 # time after death level keeps tickin'
crashed = death_delay # start with a completed crash cycle to initalize everything

# initialize keyboard object to take in control data and feed it to the player object
keyboard = Keyboard()

while True: # play the game

##############################################################################
    # Handle player death and level reset
    
    if crashed or not screen.overlap(character): # player off screen = dead!
        if crashed == death_delay:
            character = Player(level.player_start)
            crashed = False
            level.reset()
            character.current_status = Status(lives = 14, fruit = 85, boxes = 0) # arbitrary start for now
            keyboard.reset()
        else:    
            crashed += 1
            
        if crashed == 2: # just died, play ouch sound!
            ouch_sound()


##############################################################################
    # Note keyboard inputs and store them (including killing player by quitting)
    
    keyboard.handle_events(pygame.event.get())
      
    if keyboard.crashed and not crashed:
        crashed = True

##############################################################################    
    # Move every non-player object and resolve their interactions
    
    level.move_objects(land_sound = thud_sound) # input is sound boxes make when they hit the ground
    

##############################################################################
    # [if player is alive] Evolve its state, move it, and resolve interactions with other bodies

    if not crashed: # if player is dead, skip this part
          
        # determine player's state and velocity, then move
        character.evolve(keyboard)
        character.move()
          
        # sort box_list so that closest bodies interact first: prevents e.g. diagonally breaking boxes
        level.box_list = sorted(level.box_list, key= lambda x: sum((character.pos-x.pos)**2), reverse=False)
        
       # squeeze = [[False,False],[False,False]]
     
        # all objects interact with the player (destroy boxes, get fruit, land on platforms, etc)
        for i in level.box_list+level.platform_list+level.gettable_list:
            i.interact(character)
            
        # some of these interactions might have killed ya. Then ya dead!
        crashed = character.protection < 0
        
        
##############################################################################
    # Prepare canvas and draw visuals
       
    # move all non-corporeal objects to the foreground
    for body_list in [level.box_list, level.gettable_list]:
        for bod in body_list[::-1]: # need to go in reverse else removal of two objects doesn't work
            if not bod.corporeal:
                body_list.remove(bod)
                level.foreground_list.append(bod)


    # fill display with sky color and set center position for screen (only overlapping objects will draw)
    gameDisplay.fill(sky)
    screen.pos = np.array([character.pos[0]-display_size[0]/2,0])
   
    # draw all bodies, then clean foreground list of dead objects
    level.draw_level(gameDisplay,screen,character)

  #  sleep(0.1) # turned on for debugging
    pygame.display.update()
    clock.tick(45)

##############################################################################
    
pygame.quit()

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
2
3

KeyboardInterrupt: 

To Do:

Add spring boxes (metal/wood, one class that adds a shape of an arrow, and that's it, rest will be handled outside)

Add Tnt destruct sequence

Level saving

Game structure, saving, scores, etc.

Bad guys!

### Squeezing is not currently configured at all (to kill player) (is this needed?)
#### Definitely need to fix: currently, falling non-breakable boxes do not kill player

Known bugs:
falling stacks of boxes ON a moving platform results in slight overlap (but nothing functionally problematic)

Credits:
“Sound effects obtained from zapsplat.com and freesound.org"
"Idea inspired by but unrelated to Crash Bandicoot"
